**Connect and authorize google drive with google colab:**

In [ ]:
import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, ZeroPadding2D, BatchNormalization, MaxPool2D, Add, Concatenate
from yolov3.utils import read_class_names
from yolov3.configs import *

STRIDES = np.array(YOLO_STRIDES)
ANCHORS = (np.array(YOLO_ANCHORS).T/STRIDES).T

class BatchNormalization(BatchNormalization):
    # "Frozen state" and "inference mode" are two separate concepts.
    # `layer.trainable = False` is to freeze the layer, so the layer will use
    # stored moving `var` and `mean` in the "inference mode", and both `gama`
    # and `beta` will not be updated !
    def call(self, x, training=False):
        if not training:
            training = tf.constant(False)
        training = tf.logical_and(training, self.trainable)
        return super().call(x, training)

def convolutional(input_layer, filters_shape, downsample=False, activate=True, bn=True):
    if downsample:
        input_layer = ZeroPadding2D(((1, 0), (1, 0)))(input_layer)
        padding = 'valid'
        strides = 2
    else:
        strides = 1
        padding = 'same'

    conv = Conv2D(filters=filters_shape[-1], kernel_size = filters_shape[0], strides=strides,
                  padding=padding, use_bias=not bn)(input_layer)
    if bn:
        conv = BatchNormalization()(conv)
    if activate == True:
        conv = LeakyReLU(alpha=0.1)(conv)

    return conv

def residual_block(input_layer, input_channel, filter_num1, filter_num2):
    short_cut = input_layer
    conv = convolutional(input_layer, filters_shape=(1, 1, input_channel, filter_num1))
    conv = convolutional(conv       , filters_shape=(3, 3, filter_num1,   filter_num2))

    residual_output = Add()([short_cut, conv])
    return residual_output

def upsample(input_layer):
    return tf.image.resize(input_layer, (input_layer.shape[1] * 2, input_layer.shape[2] * 2), method='nearest')


def darknet53(input_data):
    input_data = convolutional(input_data, (3, 3,  3,  32))
    input_data = convolutional(input_data, (3, 3, 32,  64), downsample=True)

    for i in range(1):
        input_data = residual_block(input_data,  64,  32, 64)

    input_data = convolutional(input_data, (3, 3,  64, 128), downsample=True)

    for i in range(2):
        input_data = residual_block(input_data, 128,  64, 128)

    input_data = convolutional(input_data, (3, 3, 128, 256), downsample=True)

    for i in range(8):
        input_data = residual_block(input_data, 256, 128, 256)

    route_1 = input_data
    input_data = convolutional(input_data, (3, 3, 256, 512), downsample=True)

    for i in range(8):
        input_data = residual_block(input_data, 512, 256, 512)

    route_2 = input_data
    input_data = convolutional(input_data, (3, 3, 512, 1024), downsample=True)

    for i in range(4):
        input_data = residual_block(input_data, 1024, 512, 1024)

    return route_1, route_2, input_data

def YOLOv3(input_layer, NUM_CLASS):
    route_1, route_2, conv = darknet53(input_layer)

    conv = convolutional(conv, (1, 1, 1024,  512))
    conv = convolutional(conv, (3, 3,  512, 1024))
    conv = convolutional(conv, (1, 1, 1024,  512))
    conv = convolutional(conv, (3, 3,  512, 1024))
    conv = convolutional(conv, (1, 1, 1024,  512))
    conv_lobj_branch = convolutional(conv, (3, 3, 512, 1024))
    
    # conv_lbbox is used to predict large-sized objects , Shape = [None, 13, 13, 255] 
    conv_lbbox = convolutional(conv_lobj_branch, (1, 1, 1024, 3*(NUM_CLASS + 5)), activate=False, bn=False)

    conv = convolutional(conv, (1, 1,  512,  256))
    # upsample here uses the nearest neighbor interpolation method, which has the advantage that the
    # upsampling process does not need to learn, thereby reducing the network parameter  
    conv = upsample(conv)

    conv = Concatenate()([conv, route_2])
    conv = convolutional(conv, (1, 1, 768, 256))
    conv = convolutional(conv, (3, 3, 256, 512))
    conv = convolutional(conv, (1, 1, 512, 256))
    conv = convolutional(conv, (3, 3, 256, 512))
    conv = convolutional(conv, (1, 1, 512, 256))
    conv_mobj_branch = convolutional(conv, (3, 3, 256, 512))

    # conv_mbbox is used to predict medium-sized objects, shape = [None, 26, 26, 255]
    conv_mbbox = convolutional(conv_mobj_branch, (1, 1, 512, 3*(NUM_CLASS + 5)), activate=False, bn=False)

    conv = convolutional(conv, (1, 1, 256, 128))
    conv = upsample(conv)

    conv = Concatenate()([conv, route_1])
    conv = convolutional(conv, (1, 1, 384, 128))
    conv = convolutional(conv, (3, 3, 128, 256))
    conv = convolutional(conv, (1, 1, 256, 128))
    conv = convolutional(conv, (3, 3, 128, 256))
    conv = convolutional(conv, (1, 1, 256, 128))
    conv_sobj_branch = convolutional(conv, (3, 3, 128, 256))
    
    # conv_sbbox is used to predict small size objects, shape = [None, 52, 52, 255]
    conv_sbbox = convolutional(conv_sobj_branch, (1, 1, 256, 3*(NUM_CLASS +5)), activate=False, bn=False)

    print("conv_sbbox", conv_sbbox.name, conv_sbbox.shape)
    print("conv_mbbox", conv_mbbox.name, conv_mbbox.shape)
    print("conv_lbbox", conv_lbbox.name, conv_lbbox.shape)
    return [conv_sbbox, conv_mbbox, conv_lbbox]

def Create_Yolov3(input_size=416, channels=3, training=False, CLASSES=YOLO_COCO_CLASSES):
    NUM_CLASS = len(read_class_names(CLASSES))
    input_layer  = Input([input_size, input_size, channels])

    conv_tensors = YOLOv3(input_layer, NUM_CLASS)

    output_tensors = []
    for i, conv_tensor in enumerate(conv_tensors):
        pred_tensor = decode(conv_tensor, NUM_CLASS, i)
        output_tensors.append(pred_tensor)

    YoloV3 = tf.keras.Model(input_layer, output_tensors)
    return YoloV3

def decode(conv_output, NUM_CLASS, i=0):
    # where i = 0, 1 or 2 to correspond to the three grid scales  
    conv_shape       = tf.shape(conv_output)
    batch_size       = conv_shape[0]
    output_size      = conv_shape[1]

    conv_output = tf.reshape(conv_output, (batch_size, output_size, output_size, 3, 5 + NUM_CLASS))

    conv_raw_dxdy = conv_output[:, :, :, :, 0:2] # offset of center position     
    conv_raw_dwdh = conv_output[:, :, :, :, 2:4] # Prediction box length and width offset
    conv_raw_conf = conv_output[:, :, :, :, 4:5] # confidence of the prediction box
    conv_raw_prob = conv_output[:, :, :, :, 5: ] # category probability of the prediction box 

    # next need Draw the grid. Where output_size is equal to 13, 26 or 52  
    y = tf.range(output_size, dtype=tf.int32)
    y = tf.expand_dims(y, -1)
    y = tf.tile(y, [1, output_size])
    x = tf.range(output_size,dtype=tf.int32)
    x = tf.expand_dims(x, 0)
    x = tf.tile(x, [output_size, 1])

    xy_grid = tf.concat([x[:, :, tf.newaxis], y[:, :, tf.newaxis]], axis=-1)
    xy_grid = tf.tile(xy_grid[tf.newaxis, :, :, tf.newaxis, :], [batch_size, 1, 1, 3, 1])
    xy_grid = tf.cast(xy_grid, tf.float32)

    # Calculate the center position of the prediction box:
    pred_xy = (tf.sigmoid(conv_raw_dxdy) + xy_grid) * STRIDES[i]
    # Calculate the length and width of the prediction box:
    pred_wh = (tf.exp(conv_raw_dwdh) * ANCHORS[i]) * STRIDES[i]

    pred_xywh = tf.concat([pred_xy, pred_wh], axis=-1)
    pred_conf = tf.sigmoid(conv_raw_conf) # object box calculates the predicted confidence
    pred_prob = tf.sigmoid(conv_raw_prob) # calculating the predicted probability category box object

    # calculating the predicted probability category box object
    return tf.concat([pred_xywh, pred_conf, pred_prob], axis=-1)


In [ ]:
#================================================================
#
#   File name   : utils.py
#   Author      : PyLessons
#   Created date: 2020-09-27
#   Website     : https://pylessons.com/
#   GitHub      : https://github.com/pythonlessons/TensorFlow-2.x-YOLOv3
#   Description : additional yolov3 and yolov4 functions
#
#================================================================
from multiprocessing import Process, Queue, Pipe
import cv2
import time
import random
import colorsys
import numpy as np
import tensorflow as tf
from yolov3.configs import *
from tensorflow.python.saved_model import tag_constants

def image_preprocess(image, target_size, gt_boxes=None):
    ih, iw    = target_size
    h,  w, _  = image.shape

    scale = min(iw/w, ih/h)
    nw, nh  = int(scale * w), int(scale * h)
    image_resized = cv2.resize(image, (nw, nh))

    image_paded = np.full(shape=[ih, iw, 3], fill_value=128.0)
    dw, dh = (iw - nw) // 2, (ih-nh) // 2
    image_paded[dh:nh+dh, dw:nw+dw, :] = image_resized
    image_paded = image_paded / 255.

    if gt_boxes is None:
        return image_paded

    else:
        gt_boxes[:, [0, 2]] = gt_boxes[:, [0, 2]] * scale + dw
        gt_boxes[:, [1, 3]] = gt_boxes[:, [1, 3]] * scale + dh
        return image_paded, gt_boxes


def draw_bbox(image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence = True, Text_colors=(255,255,0), rectangle_colors='', tracking=False):   
    NUM_CLASS = read_class_names(CLASSES)
    num_classes = len(NUM_CLASS)
    image_h, image_w, _ = image.shape
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    #print("hsv_tuples", hsv_tuples)
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    for i, bbox in enumerate(bboxes):
        coor = np.array(bbox[:4], dtype=np.int32)
        score = bbox[4]
        class_ind = int(bbox[5])
        bbox_color = rectangle_colors if rectangle_colors != '' else colors[class_ind]
        bbox_thick = int(0.6 * (image_h + image_w) / 1000)
        if bbox_thick < 1: bbox_thick = 1
        fontScale = 0.75 * bbox_thick
        (x1, y1), (x2, y2) = (coor[0], coor[1]), (coor[2], coor[3])

        # put object rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), bbox_color, bbox_thick*2)

        if show_label:
            # get text label
            score_str = " {:.2f}".format(score) if show_confidence else ""

            if tracking: score_str = " "+str(score)

            try:
                label = "{}".format(NUM_CLASS[class_ind]) + score_str
            except KeyError:
                print("You received KeyError, this might be that you are trying to use yolo original weights")
                print("while using custom classes, if using custom model in configs.py set YOLO_CUSTOM_WEIGHTS = True")

            # get text size
            (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                                  fontScale, thickness=bbox_thick)
            # put filled text rectangle
            cv2.rectangle(image, (x1, y1), (x1 + text_width, y1 - text_height - baseline), bbox_color, thickness=cv2.FILLED)

            # put text above rectangle
            cv2.putText(image, label, (x1, y1-4), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale, Text_colors, bbox_thick, lineType=cv2.LINE_AA)

    return image


def bboxes_iou(boxes1, boxes2):
    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)

    boxes1_area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
    boxes2_area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])

    left_up       = np.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down    = np.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = np.maximum(right_down - left_up, 0.0)
    inter_area    = inter_section[..., 0] * inter_section[..., 1]
    union_area    = boxes1_area + boxes2_area - inter_area
    ious          = np.maximum(1.0 * inter_area / union_area, np.finfo(np.float32).eps)

    return ious


def nms(bboxes, iou_threshold, sigma=0.3, method='nms'):
    """
    :param bboxes: (xmin, ymin, xmax, ymax, score, class)

    Note: soft-nms, https://arxiv.org/pdf/1704.04503.pdf
          https://github.com/bharatsingh430/soft-nms
    """
    classes_in_img = list(set(bboxes[:, 5]))
    best_bboxes = []

    for cls in classes_in_img:
        cls_mask = (bboxes[:, 5] == cls)
        cls_bboxes = bboxes[cls_mask]
        # Process 1: Determine whether the number of bounding boxes is greater than 0 
        while len(cls_bboxes) > 0:
            # Process 2: Select the bounding box with the highest score according to socre order A
            max_ind = np.argmax(cls_bboxes[:, 4])
            best_bbox = cls_bboxes[max_ind]
            best_bboxes.append(best_bbox)
            cls_bboxes = np.concatenate([cls_bboxes[: max_ind], cls_bboxes[max_ind + 1:]])
            # Process 3: Calculate this bounding box A and
            # Remain all iou of the bounding box and remove those bounding boxes whose iou value is higher than the threshold 
            iou = bboxes_iou(best_bbox[np.newaxis, :4], cls_bboxes[:, :4])
            weight = np.ones((len(iou),), dtype=np.float32)

            assert method in ['nms', 'soft-nms']

            if method == 'nms':
                iou_mask = iou > iou_threshold
                weight[iou_mask] = 0.0

            if method == 'soft-nms':
                weight = np.exp(-(1.0 * iou ** 2 / sigma))

            cls_bboxes[:, 4] = cls_bboxes[:, 4] * weight
            score_mask = cls_bboxes[:, 4] > 0.
            cls_bboxes = cls_bboxes[score_mask]

    return best_bboxes


def postprocess_boxes(pred_bbox, original_image, input_size, score_threshold):
    valid_scale=[0, np.inf]
    pred_bbox = np.array(pred_bbox)

    pred_xywh = pred_bbox[:, 0:4]
    pred_conf = pred_bbox[:, 4]
    pred_prob = pred_bbox[:, 5:]

    # 1. (x, y, w, h) --> (xmin, ymin, xmax, ymax)
    pred_coor = np.concatenate([pred_xywh[:, :2] - pred_xywh[:, 2:] * 0.5,
                                pred_xywh[:, :2] + pred_xywh[:, 2:] * 0.5], axis=-1)
    # 2. (xmin, ymin, xmax, ymax) -> (xmin_org, ymin_org, xmax_org, ymax_org)
    org_h, org_w = original_image.shape[:2]
    resize_ratio = min(input_size / org_w, input_size / org_h)

    dw = (input_size - resize_ratio * org_w) / 2
    dh = (input_size - resize_ratio * org_h) / 2

    pred_coor[:, 0::2] = 1.0 * (pred_coor[:, 0::2] - dw) / resize_ratio
    pred_coor[:, 1::2] = 1.0 * (pred_coor[:, 1::2] - dh) / resize_ratio

    # 3. clip some boxes those are out of range
    pred_coor = np.concatenate([np.maximum(pred_coor[:, :2], [0, 0]),
                                np.minimum(pred_coor[:, 2:], [org_w - 1, org_h - 1])], axis=-1)
    invalid_mask = np.logical_or((pred_coor[:, 0] > pred_coor[:, 2]), (pred_coor[:, 1] > pred_coor[:, 3]))
    pred_coor[invalid_mask] = 0

    # 4. discard some invalid boxes
    bboxes_scale = np.sqrt(np.multiply.reduce(pred_coor[:, 2:4] - pred_coor[:, 0:2], axis=-1))
    scale_mask = np.logical_and((valid_scale[0] < bboxes_scale), (bboxes_scale < valid_scale[1]))

    # 5. discard boxes with low scores
    classes = np.argmax(pred_prob, axis=-1)
    scores = pred_conf * pred_prob[np.arange(len(pred_coor)), classes]
    score_mask = scores > score_threshold
    mask = np.logical_and(scale_mask, score_mask)
    coors, scores, classes = pred_coor[mask], scores[mask], classes[mask]

    return np.concatenate([coors, scores[:, np.newaxis], classes[:, np.newaxis]], axis=-1)


def detect_image(Yolo, original_image, input_size=416, CLASSES=YOLO_COCO_CLASSES, score_threshold=0.3, iou_threshold=0.45, rectangle_colors=''):
    image_data = image_preprocess(np.copy(original_image), [input_size, input_size])
    image_data = image_data[np.newaxis, ...].astype(np.float32)
    
    pred_bbox = Yolo.predict(image_data)
        
    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    
    bboxes = postprocess_boxes(pred_bbox, original_image, input_size, score_threshold)
    #bboxes = nms(bboxes, iou_threshold, method='nms')
    print(bboxes)
    image = draw_bbox(original_image, bboxes, CLASSES=CLASSES, rectangle_colors=rectangle_colors)
        
    return image


**Test by loading trained model:**

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
from yolov3.configs import *
import random

label_txt = "objects/objects_test.txt"
labels = open(label_txt).readlines()
index = 18
model = Create_Yolov3(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)
model.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}") # use keras weights

In [ ]:
image_info = labels[index].split()

image_path = image_info[0]
original_image      = cv2.imread(image_path) * 2

image = detect_image(model, original_image, input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0), iou_threshold = 0.1)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,15))
plt.imshow(image)

**Test by testing detection on original model:**

In [ ]:
import numpy as np
import os
from tensorflow.keras.layers import Activation, Concatenate, Add, ZeroPadding2D, LeakyReLU, BatchNormalization, Conv2D, Input, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU, InputLayer, Conv2DTranspose, DepthwiseConv2D

model_dir = "A:/models/yolo/"
model_name = "yolo"

epsilon = np.finfo(float).eps
lines_in_graph = []

model_data_dir = os.path.join(model_dir, model_name)
if not os.path.exists(model_data_dir):
    os.makedirs(model_data_dir)
    
for i, layer in enumerate(model.layers):
    print(i, ":", layer.__class__.__name__, layer.name)
    items_to_write = [layer.__class__.__name__, layer.name]
    if isinstance(layer,InputLayer):
        inpu_shape = layer.input.shape
        items_to_write.append("")
        items_to_write.append(str(inpu_shape[2]))
        items_to_write.append(str(inpu_shape[1]))
        items_to_write.append(str(inpu_shape[3]))
    elif isinstance(layer.input, list):
        input_layer_names = []
        for i in range(len(layer.input)):
            inpu_layer_name = layer.input[i].name.split('/')[0]
            input_layer_names.append(inpu_layer_name)
        items_to_write.append("&".join(input_layer_names))
        if isinstance(layer, Add):
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,Concatenate): 
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
            '''
            elif isinstance(layer,HeadWrapper):
                for i in range(len(layer.input)):
                    inpu_shape = layer.input[i].shape
                    items_to_write.append(str(inpu_shape[2]))
                    items_to_write.append(str(inpu_shape[1]))
                    items_to_write.append(str(inpu_shape[3])) '''
        else:
            print("---------------------------------")
            print("Error: Multi input layer not known")
            print("---------------------------------")            
    else:
        inpu_layer_name = layer.input.name.split('/')[0]
        items_to_write.append(inpu_layer_name)
        if isinstance(layer,Conv2D):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            layer_activation = layer.activation
            if layer_activation:
                items_to_write.append(layer_activation.__name__)
            else:
                print("Warning: Activation layer present : ", layer_activation.__name__)                
                print("---------------------------------")
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            dilation_rates = layer.dilation_rate
            if dilation_rates[0] != dilation_rates[1] and dilation_rates[1] != 1:
                print("---------------------------------")
                print("Error: dilation_rates must be equal")
                print("---------------------------------")                
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
            items_to_write.append(str(layer.padding))            
        elif isinstance(layer,Conv2DTranspose):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,DepthwiseConv2D):   
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,BatchNormalization):
            weights = layer.get_weights()
            gamma = weights[0]
            beta = weights[1]
            moving_mean = weights[2]
            moving_variance = weights[3]
            valueToSqrt = np.sqrt(moving_variance + epsilon)
            a = gamma / np.sqrt(valueToSqrt)
            b = - moving_mean * a + beta
            path_to_save = os.path.join(model_data_dir, layer.name + '_mean.npy')
            np.save(path_to_save, b)
            path_to_save = os.path.join(model_data_dir, layer.name + '_variance.npy')
            np.save(path_to_save, a) 

            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,MaxPooling2D):
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,ZeroPadding2D):
            if(layer.padding[0][1] != layer.padding[1][1] and layer.padding[0][0] == 0 and layer.padding[1][0] != 0):
                print(layer.padding)
                print("---------------------------------")
                print("Error: This padding not supported") 
                print("---------------------------------")
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(layer.padding[0][0]))
            items_to_write.append(str(layer.padding[0][1]))
            items_to_write.append(str(layer.padding[1][0]))
            items_to_write.append(str(layer.padding[1][1]))
        elif isinstance(layer, ReLU):
            maxValue = layer.max_value
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(maxValue))
        elif isinstance(layer, LeakyReLU):
            alpha = layer.alpha.item()
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(alpha))
        elif isinstance(layer, Activation):
            activation_type = layer.get_config()['activation']
            maxValue = 0
            if activation_type == 'relu':
                items_to_write.append("relu")
                maxValue = 0
                if hasattr(layer,'max_value'):
                    maxValue = layer.max_value
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
                items_to_write.append(str(maxValue))
            elif activation_type == 'sigmoid':
                items_to_write.append("sigmoid")
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
            elif activation_type == 'softmax':
                items_to_write.append("softmax")
                inpu_shape = layer.input.shape
                output_shape = layer.output.shape
                print(output_shape)
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[2]))
            else:
                print("---------------------------------")
                print("Error: Only Relu activation with max value supported, provided is ", activation_type)
                print("---------------------------------") 
        else:
            print(layer.input.shape)
            print("Error: Layer is not identified ", layer.__class__.__name__, "-> ", layer.name)
    lines_in_graph.append(",".join(items_to_write))
with open(os.path.join(model_dir, f'{model_name}.txt'), 'w') as f:
    # Write each line of the list to the file
    for line in lines_in_graph:            
        f.write(line + '\n')
    print("File is saved to " + f'{model_name}.txt')    